# Warcraft Lore Analysis

This notebook analyzes Warcraft lore through web scraping and text analysis to identify thematic patterns in character narratives. We'll examine themes like duty, power, sacrifice, corruption, and freedom across major characters like Arthas, Illidan, and Thrall.

## 📋 Notebook Overview

**Purpose:** Computational analysis of philosophical archetypes (Stoic, Cynic, Platonic Tyrant) in Warcraft transmedia narratives.

**What This Does:**
- Scrapes Warcraft lore from Wowpedia and (optionally) Wowhead quest databases
- Extracts passages containing keywords: duty, power, sacrifice, corruption, freedom
- Analyzes keyword distribution across Arthas, Illidan, and Thrall
- Exports data to CSV for qualitative coding in your paper analysis
- Generates visualizations and clusters thematic passages

**✅ To Run:** Simply execute "Run All" (Ctrl+Shift+F10) and the notebook will complete all steps automatically.

**⏱️ Estimated Time:** 5-10 minutes for Wowpedia scraping, 30+ minutes if Selenium/Wowhead is enabled.

In [ ]:
# === SETUP: Imports, Configuration, and Function Definitions ===
print("=" * 60)
print("WARCRAFT LORE ANALYSIS - Philosophical Archetype Extraction")
print("=" * 60)
print("\n[1/4] Loading libraries...")

# 1. Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from collections import Counter
import PyPDF2
import time
import matplotlib.pyplot as plt

print("✓ Core libraries loaded")

# 2. Configuration
KEYWORDS = ['duty', 'power', 'sacrifice', 'corruption', 'freedom']
URLS = {
    'arthas': ['https://wowpedia.fandom.com/wiki/Arthas_Menethil'],
    'illidan': ['https://wowpedia.fandom.com/wiki/Illidan_Stormrage'],
    'thrall': ['https://wowpedia.fandom.com/wiki/Thrall']
}
NOVELS = ['arthas_rise.pdf', 'illidan.pdf', 'lord_clans.pdf']
USE_PDFS = False

# Optional: Enable Wowhead scraping (slow, may not work due to JavaScript/HTML changes)
# Set to True only if you need comprehensive quest data and have 30+ minutes
USE_WOWHEAD = False

print(f"✓ Configuration loaded (Keywords: {', '.join(KEYWORDS)})")
print(f"  Characters: Arthas, Illidan, Thrall")
print(f"  PDF novels: {'Enabled' if USE_PDFS else 'Disabled'}")
print(f"  Wowhead quests: {'Enabled' if USE_WOWHEAD else 'Disabled (use Wowpedia only)'}")

# 3. Define core scraping functions
print("\n[2/4] Defining analysis functions...")

def scrape_page(url, char):
    """Scrape Wowpedia page for keyword-containing passages."""
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    texts = [p.text for p in soup.find_all('p') if any(kw.lower() in p.text.lower() for kw in KEYWORDS)]
    return [{'char': char, 'source': url, 'text': t, 'keywords': [kw for kw in KEYWORDS if kw.lower() in t.lower()]} for t in texts]

def extract_novel(pdf_path, char):
    """Extract keyword passages from PDF novels."""
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        texts = []
        for page in reader.pages:
            text = page.extract_text()
            if any(kw.lower() in text.lower() for kw in KEYWORDS):
                texts.append({
                    'char': char, 
                    'source': pdf_path, 
                    'text': text[:500], 
                    'keywords': [kw for kw in KEYWORDS if kw.lower() in text.lower()]
                })
    return texts

def extract_context(text, keyword, window=50):
    """Extract context around keyword occurrences (±50 chars)."""
    kw_pos = [m.start() for m in re.finditer(re.escape(keyword), text, re.IGNORECASE)]
    contexts = []
    for pos in kw_pos:
        start = max(0, pos - window)
        end = min(len(text), pos + window + len(keyword))
        contexts.append(text[start:end])
    return contexts

print("✓ Wowpedia scraper defined")
print("✓ PDF parser defined")
print("✓ Context extraction defined")

# 4. Check Selenium availability (optional for Wowhead)
print("\n[3/4] Checking optional dependencies...")
try:
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.chrome.options import Options
    from webdriver_manager.chrome import ChromeDriverManager
    
    def scrape_wowhead_with_selenium(chars, expansions=['wotlk']):
        """Scrape Wowhead quests using Selenium (handles JavaScript rendering)."""
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--log-level=3')  # Suppress logs
        
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        data = []
        
        try:
            for exp in expansions:
                for char in chars:
                    url = f'https://www.wowhead.com/{exp}/quests?filter=cr=160;cs=1;str={char.lower()}'
                    print(f"  Scraping {exp}/{char}...")
                    
                    driver.get(url)
                    time.sleep(5)
                    
                    quest_links = driver.find_elements(By.CSS_SELECTOR, 'a.q, a.q2, a.q1')
                    
                    for link in quest_links[:20]:
                        try:
                            title = link.text
                            quest_url = link.get_attribute('href')
                            
                            if not quest_url or 'quest=' not in quest_url:
                                continue
                            
                            driver.get(quest_url)
                            time.sleep(2)
                            
                            text = ""
                            for selector in ['.text', '.objective', '.q-description']:
                                try:
                                    elem = driver.find_element(By.CSS_SELECTOR, selector)
                                    text += elem.text + " "
                                except:
                                    pass
                            
                            if text and any(kw.lower() in text.lower() for kw in KEYWORDS):
                                data.append({
                                    'char': char,
                                    'expansion': exp,
                                    'source': quest_url,
                                    'title': title,
                                    'text': text[:1500],
                                    'keywords': [kw for kw in KEYWORDS if kw.lower() in text.lower()]
                                })
                            
                            time.sleep(1)
                        except Exception:
                            continue
        finally:
            driver.quit()
        
        return pd.DataFrame(data)
    
    SELENIUM_AVAILABLE = True
    print("✓ Selenium available (Wowhead scraping enabled)")
except ImportError:
    SELENIUM_AVAILABLE = False
    print("⚠ Selenium not installed (Wowhead scraping disabled)")
    print("  Install with: pip install selenium webdriver-manager")

print("\n[4/4] Setup complete! Starting data collection...\n")

## Setup & Configuration

This cell loads all libraries, defines scraping functions, and checks for optional dependencies.

## Data Collection: Wowpedia

Scrapes character pages for passages containing the five philosophical keywords.

## 5. Data Collection and Scraping

In [ ]:
# === DATA COLLECTION: Scrape Wowpedia ===
print("=" * 60)
print("STEP 1: WOWPEDIA SCRAPING")
print("=" * 60)

data = []

# Scrape Wowpedia character pages
print("\nScraping Wowpedia for thematic passages...")
for char, urls in URLS.items():
    for url in urls:
        if url.startswith('http'):
            try:
                scraped = scrape_page(url, char)
                data.extend(scraped)
                print(f"  ✓ {char.capitalize()}: {len(scraped)} passages from Wowpedia")
            except Exception as e:
                print(f"  ✗ Error scraping {char}: {e}")

# Parse PDF novels (if enabled)
if USE_PDFS:
    print("\nParsing PDF novels...")
    novel_mappings = [
        ('arthas_rise.pdf', 'arthas'),
        ('illidan.pdf', 'illidan'), 
        ('lord_clans.pdf', 'thrall')
    ]
    
    for novel, char in novel_mappings:
        try:
            novel_data = extract_novel(novel, char)
            data.extend(novel_data)
            print(f"  ✓ {char.capitalize()}: {len(novel_data)} passages from {novel}")
        except FileNotFoundError:
            print(f"  ⚠ {novel} not found (skipping)")
else:
    print("\n  PDF parsing disabled (set USE_PDFS=True to enable)")

# Extract context snippets around keywords
print("\nExtracting keyword contexts...")
for passage in data:
    context_passages = []
    for kw in KEYWORDS:
        context_passages.extend(extract_context(passage['text'], kw))
    passage['context'] = ' | '.join(set(context_passages))

print(f"\n✓ Wowpedia collection complete: {len(data)} total passages")
print("=" * 60 + "\n")

In [ ]:
# === DATA COLLECTION: Scrape Wowhead (Optional) ===
print("=" * 60)
print("STEP 2: WOWHEAD QUEST SCRAPING (OPTIONAL)")
print("=" * 60)

if USE_WOWHEAD and SELENIUM_AVAILABLE:
    print("\nSelenium enabled - scraping Wowhead quest database...")
    print("⏱️ This may take 10-30 minutes (JavaScript rendering + rate limiting)\n")
    
    try:
        selenium_quests = scrape_wowhead_with_selenium(['Thrall', 'Arthas'], expansions=['wotlk'])
        
        if len(selenium_quests) > 0:
            selenium_quests['count'] = selenium_quests['keywords'].apply(len)
            selenium_quests.to_csv('wowhead_quests_selenium.csv', index=False)
            
            print(f"\n✓ Wowhead collection complete: {len(selenium_quests)} quests")
            print("\nKeyword distribution by character/expansion:")
            print(selenium_quests.groupby(['char', 'expansion'])['count'].sum())
        else:
            print("\n⚠ No quests collected from Wowhead")
            print("  Possible causes: JavaScript timeout, changed HTML structure, or no keyword matches")
    except Exception as e:
        print(f"\n✗ Wowhead scraping failed: {e}")
        print("  Continuing with Wowpedia data only...")
else:
    if not USE_WOWHEAD:
        print("\n⏭️ Wowhead scraping disabled (USE_WOWHEAD=False)")
        print("  Set USE_WOWHEAD=True in the Setup cell to enable quest scraping")
    elif not SELENIUM_AVAILABLE:
        print("\n⚠ Selenium not installed - Wowhead scraping unavailable")
        print("  Install with: pip install selenium webdriver-manager")
    print("  Continuing with Wowpedia data only...")
    print("  To enable: pip install selenium webdriver-manager")

print("\n" + "=" * 60 + "\n")

## Data Collection: Wowhead (Optional)

Uses Selenium to scrape quest text from Wowhead's JavaScript-rendered pages. Skipped if Selenium unavailable.

## Keyword Analysis & Visualization

Analyzes keyword frequency across characters and generates visualizations for your paper.

In [ ]:
# === ANALYSIS: Keyword Distribution & Visualization ===
print("=" * 60)
print("STEP 3: KEYWORD ANALYSIS & VISUALIZATION")
print("=" * 60)

df = pd.DataFrame(data)

if len(df) > 0:
    print(f"\nAnalyzing {len(df)} passages...")
    
    # Calculate keyword frequency
    df['count'] = df['keywords'].apply(len)
    dense_passages = df[df['count'] >= 1].head(100)
    
    # Export for qualitative coding
    dense_passages.to_csv('warcraft_passages.csv', index=False)
    print(f"✓ Exported {len(dense_passages)} passages → warcraft_passages.csv")
    
    # Generate visualization
    print("\nGenerating keyword frequency chart...")
    char_counts = dense_passages.groupby('char')['count'].sum()
    
    fig, ax = plt.subplots(figsize=(10, 6))
    char_counts.plot.bar(ax=ax, color=['#C41E3A', '#0070DE', '#8B4513'])
    ax.set_title('Keyword Frequency by Character (Duty, Power, Sacrifice, Corruption, Freedom)', 
                 fontsize=14, fontweight='bold')
    ax.set_ylabel('Total Keyword Count', fontsize=12)
    ax.set_xlabel('Character', fontsize=12)
    ax.tick_params(axis='x', rotation=45)
    plt.tight_layout()
    plt.savefig('keyword_frequency.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print summary statistics
    print("\n" + "=" * 40)
    print("KEYWORD FREQUENCY BY CHARACTER")
    print("=" * 40)
    for char, count in char_counts.items():
        print(f"  {char.capitalize():12} : {count:3} keyword occurrences")
    print("=" * 40)
    
    # Keyword breakdown
    print("\n" + "=" * 40)
    print("KEYWORD DISTRIBUTION")
    print("=" * 40)
    all_keywords = [kw for keywords in df['keywords'] for kw in keywords]
    keyword_counts = Counter(all_keywords)
    for kw, count in keyword_counts.most_common():
        print(f"  {kw.capitalize():15} : {count:3} occurrences")
    print("=" * 40)
    
else:
    print("\n✗ No data collected - check URLs and connection")

print("\n" + "=" * 60 + "\n")

## Thematic Clustering

Groups passages into clusters to identify philosophical patterns (Stoic duty vs. Cynic freedom vs. Tyrant power).

In [ ]:
# === ANALYSIS: Text Clustering (Thematic Patterns) ===
print("=" * 60)
print("STEP 4: THEMATIC CLUSTERING ANALYSIS")
print("=" * 60)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

if len(dense_passages) > 0:
    print(f"\nClustering {len(dense_passages)} passages into 3 thematic groups...")
    
    # Vectorize text using TF-IDF
    vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
    X = vectorizer.fit_transform(dense_passages['text'])
    
    # Apply K-means clustering
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    dense_passages['cluster'] = kmeans.fit_predict(X)
    
    print("✓ Clustering complete\n")
    
    # Display sample passages from each cluster
    print("=" * 60)
    print("SAMPLE PASSAGES BY THEMATIC CLUSTER")
    print("=" * 60)
    
    for cluster_id in range(3):
        print(f"\n--- CLUSTER {cluster_id} ---")
        cluster_passages = dense_passages[dense_passages['cluster'] == cluster_id]
        cluster_size = len(cluster_passages)
        print(f"Size: {cluster_size} passages")
        
        if cluster_size > 0:
            # Show character distribution in cluster
            char_dist = cluster_passages['char'].value_counts()
            print(f"Characters: {dict(char_dist)}")
            
            # Show top keywords in cluster
            cluster_keywords = [kw for keywords in cluster_passages['keywords'] for kw in keywords]
            top_keywords = Counter(cluster_keywords).most_common(3)
            print(f"Top keywords: {', '.join([kw for kw, _ in top_keywords])}")
            
            # Show sample passage
            sample = cluster_passages.iloc[0]
            print(f"\nSample passage ({sample['char']}):")
            print(f"  {sample['text'][:200]}...")
    
    print("\n" + "=" * 60)
    
    # Export clustered data
    dense_passages.to_csv('warcraft_passages_clustered.csv', index=False)
    print("\n✓ Exported clustered data → warcraft_passages_clustered.csv")
    
    # Cluster distribution
    print("\nCluster distribution:")
    print(dense_passages.groupby(['cluster', 'char']).size().unstack(fill_value=0))
    
else:
    print("\n⚠ No passages available for clustering")

print("\n" + "=" * 60)
print("ANALYSIS COMPLETE")
print("=" * 60)
print("\n📊 Output files generated:")
print("  • warcraft_passages.csv - All extracted passages")
print("  • warcraft_passages_clustered.csv - With cluster assignments")
print("  • keyword_frequency.png - Visualization chart")
if SELENIUM_AVAILABLE:
    print("  • wowhead_quests_selenium.csv - Quest data (if Selenium ran)")
print("\n✅ Ready for qualitative coding in your philosophical archetype analysis!")
print("=" * 60)